# Introduction
---
In this notebook I am developing methods for MCMC sampling.

In [1]:
import sys
sys.path.append('../')
from sampler import MCMC

In [2]:
hallucination = MCMC.Hallucination(native_seq='AHVFGRACY')   
print(hallucination)

ProteusAI.MCMC.Hallucination class: 
---------------------------------------
When Hallucination.run() sequences will be hallucinated using this seed sequence:

AHVFGRACY

The following variables were set:

variable	|value
----------------+-------------------
sampler: 	|simulated_annealing
n_traj: 	|5
n_iter: 	|1000
mut_p: 		|(0.6, 0.2, 0.2)
T: 		|10.0
M: 		|0.01

The energy function is a linear combination of the following constraints:

constraint	|value	|weight
----------------+-------+------------
length 		|200	|0.2



In [6]:
import os
from subprocess import DEVNULL, STDOUT, check_call

path_to_files = '../scripts/hallucinations/'
outdir = '../scripts/pngs/'
pdbs = [path_to_files + file for file in os.listdir(path_to_files)]
pdbs = sorted(pdbs, key=lambda f: int(f.split('/')[-1].split('_hallucination')[0]))

hallucinations = {}
for pdb in pdbs:
    traj = (pdb.split('_hallucination_')[-1][:-4])
    if traj not in hallucinations.keys():
        hallucinations[traj] = [pdb]
    else:
        hallucinations[traj].append(pdb)

images = {}
for traj in hallucinations.keys():
    
    for i, pdb in enumerate(hallucinations[traj]):
        if i == 0:
            # only open this one file without alignemt
            struc = pdb
            png = outdir + pdb.split('/')[-1][:-4] + '.png'
            lines = [
                f'load {struc}, struc',
                'spectrum b, blue_white_red, minimum=20, maximum=50',
                'bg_color white',
                'set ray_opaque_background, off',
                'set ray_trace_fog, 0.5',
                'set ray_shadows, off',
                'set ambient, 0.2',
                'ray 1200, 1200',
                f'png {png}',
                'quit'
            ]
            with open('make_png.pml', 'w') as f:
                for line in lines:
                    f.writelines(line + '\n')
            
            images[traj] = [png]
            check_call(['pymol', 'make_png.pml'], stdout=DEVNULL, stderr=STDOUT)
        else:
            # align to previous file
            struc = pdb
            prev_struc = hallucinations[traj][i-1]
            png = outdir + pdb.split('/')[-1][:-4] + '.png'
            lines = [
                f'load {struc}, struc',
                f'load {prev_struc}, prev_struc',
                'align struc, prev_struc',
                'delete prev_struc',
                'spectrum b, blue_white_red, minimum=20, maximum=50',
                'bg_color white',
                'set ray_opaque_background, off',
                'set ray_trace_fog, 0.5',
                'set ray_shadows, off',
                'set ambient, 0.2',
                'ray 1200, 1200',
                f'png {png}',
                'quit'
            ]
            with open('make_png.pml', 'w') as f:
                for line in lines:
                    f.writelines(line + '\n')
            
            images[traj].append(png)
            check_call(['pymol', 'make_png.pml'], stdout=DEVNULL, stderr=STDOUT)

In [10]:
import imageio.v2 as imageio
import os

# Read each PNG file and append to the list of frames

for i in images.keys():
    frames = []
    for file in images[i]:
        frames.append(imageio.imread(file))
    

    # Save the frames as a GIF
    output_file = f"hallucination_{i}.gif"
    imageio.mimsave(output_file, frames, duration=0.5)

/var/folders/sd/pr3fylj57wlgrmcxg1v4zp3c0000gn/T/ipykernel_89984/778785286.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(file))


In [11]:
from IPython.display import HTML

# Replace these file paths with your own
gif1 = "hallucination_0.gif"
gif2 = "hallucination_1.gif"
gif3 = "hallucination_2.gif"
gif4 = "hallucination_3.gif"
gif5 = "hallucination_4.gif"

# Create the HTML code to display the GIFs in a 2x2 grid
html = f"""
<div style="display: flex; flex-wrap: wrap; justify-content: space-between;">
    <img src="{gif1}" style="width: 18%; margin-bottom: 10px;">
    <img src="{gif2}" style="width: 18%; margin-bottom: 10px;">
    <img src="{gif3}" style="width: 18%; margin-bottom: 10px;">
    <img src="{gif4}" style="width: 18%; margin-bottom: 10px;">
    <img src="{gif5}" style="width: 18%; margin-bottom: 10px;">
</div>
"""

# Display the HTML code
HTML(html)